<a href="https://colab.research.google.com/github/farghea/Pose-Detection-Biomechanics/blob/main/fit3d_metrabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import os

# Check if the data directory exists, and create it if it doesn't
if not os.path.exists("/content/data/"):
    os.makedirs("/content/data/")

In [ ]:
# Extract the tar.gz file
!tar -xzvf "/content/gdrive/MyDrive/integrating new tech/3-6million-fit3d/fit3d_train.tar.gz" -C "/content/data/"



tar (child): /content/gdrive/MyDrive/integrating new tech/3-6million-fit3d/fit3d_train.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
import shutil

# Copy all files from the source directory to the destination directory, excluding the tar.gz file
src_dir = "/content/gdrive/MyDrive/integrating new tech/3-6million-fit3d/"
dst_dir = "/content/data/"
for filename in os.listdir(src_dir):
    if  ".tar.gz" not in filename:
        shutil.copy2(os.path.join(src_dir, filename), dst_dir)



In [ ]:
import cv2

# Function to read JSON file
def read_json_file(file_path):
  import json
  with open(file_path, 'r') as file:
    data = json.load(file)
  return data

def split_videos(subject_name, task_name, camera_name, frame_number):
    dir_path_frames = f"/content/data/train/{subject_name}/videos/{camera_name}"

    # Check if the directory exists
    if not os.path.exists(dir_path_frames):
        print(f"Directory not found: {dir_path_frames}")
        return None

    video_path = f"{dir_path_frames}/{task_name}.mp4"
    # print(video_path)
    # Check if the video file exists
    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        return None

    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Set the video to the desired frame number
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)

    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Read the frame
    ret, frame = video.read()

    if ret:
        # Save the frame as a JPG image
        jpg_file_name = f"frame{frame_number}.jpg"
        cv2.imwrite(jpg_file_name, frame)
        # print(f"Frame {frame_number} saved as frame{frame_number}.jpg")
    else:
        print(f"Failed to read frame {frame_number}")

    # Release the video file
    video.release()
    return total_frames, jpg_file_name


fit3d_info = read_json_file('/content/data/fit3d_info.json');


In [ ]:
!pwd


In [ ]:
# Function to calculate some parameters from measurements

def cal_ground_truth_parameters(subject_name, task_name, frame_no):
  import numpy as np
  # Read the JSON file
  file_path = f"/content/data/train/{subject_name}/joints3d_25/{task_name}.json"
  json_data = read_json_file(file_path)

  frame_length = len(json_data['joints3d_25'])
  # print(file_path)

  points = json_data['joints3d_25'][frame_no - 1]

  # Separate x, y, and z coordinates
  x_coords = [point[0] for point in points]
  y_coords = [point[1] for point in points]
  z_coords = [point[2] for point in points]

  validation_landmark = {
      "mid-hip": [0],
      "hip": [1, 4],
      "neck": [8],
      "shoulder": [11, 14],
      "hand": [13, 16],
      "knee": [2, 5]
  }

  # knee mid points
  indx = validation_landmark['knee']
  knee_mid_point = np.array([points[indx[0]], points[indx[1]]])
  knee_mid_point = np.mean(knee_mid_point, axis = 0)

  # Flexion angle
  mid_hip_pnt = np.array(points[validation_landmark['mid-hip'][0]])
  neck_pnt = np.array(points[validation_landmark['neck'][0]])

  myvector = neck_pnt - mid_hip_pnt
  myvector = myvector/np.linalg.norm(myvector)

  myvector2 = mid_hip_pnt - knee_mid_point
  myvector2 = myvector2/np.linalg.norm(myvector2)

  flexion_angle = np.rad2deg(np.arccos(np.dot(myvector, myvector2)))

  # Shoulder length
  indx = validation_landmark['shoulder']
  shoulder_pnt = np.array([points[indx[0]], points[indx[1]]])
  myvector = shoulder_pnt[0, :] - shoulder_pnt[1, :]

  shoulder_length = np.linalg.norm(myvector)

  # hand mid point
  indx = validation_landmark['hand']
  hand_mid_point = np.array([points[indx[0]], points[indx[1]]])
  hand_mid_point = np.mean(hand_mid_point, axis = 0)

  # disntace to mid hip (normalized by shoulder length)
  distance_to_hip = mid_hip_pnt - hand_mid_point
  distance_to_hip = np.linalg.norm(distance_to_hip)/shoulder_length

  return flexion_angle, distance_to_hip, frame_length



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

model = hub.load('https://bit.ly/metrabs_l')  # Takes about 3 minutes

joint_names = model.per_skeleton_joint_names['smpl+head_30'].numpy().astype(str)
joint_edges = model.per_skeleton_joint_edges['smpl+head_30'].numpy()


In [ ]:
import numpy as np

landmark_info = {
      'hip': [1, 2],
      'shoulder': [16, 17],
      'knee': [4, 5],
      'hand': [22, 23]}

def cal_mid_point(landmark_name, pred_pose):
  indx = landmark_info[landmark_name]
  pnt1 = pred_pose[indx[0], :]
  pnt2 = pred_pose[indx[1], :]
  mid_point = 0.5*(pnt1 + pnt2)
  return mid_point

def estimate_3D_pose(jpg_file_name):
  img = tf.image.decode_image(tf.io.read_file(jpg_file_name))
  pred = model.detect_poses(img, skeleton='smpl+head_30')

  pred_pose = pred['poses3d'].numpy().reshape((30, 3))

  # lambda function to normalize a vector
  normalize = lambda vec: vec / np.linalg.norm(vec)

  # mid hip
  mid_hip = cal_mid_point('hip', pred_pose)

  # mid knee
  mid_knee = cal_mid_point('knee', pred_pose)

  # mid shoulder
  mid_shoulder = cal_mid_point('shoulder', pred_pose)

  # mid hand
  mid_hand = cal_mid_point('hand', pred_pose)

  # Shoulder length
  pnt1 = pred_pose[16, :]
  pnt2 = pred_pose[17, :]
  shoulder_length = np.linalg.norm(pnt2 - pnt1)

  # Flexion angle
  v1 = normalize(mid_shoulder - mid_hip)
  v2 = normalize(mid_hip - mid_knee)
  flexion_angle = np.rad2deg(np.arccos(np.dot(v1, v2)))

  # hand to mid-hip distance (normalized to the shoulder distance)
  mid_hip_to_hand_distance = np.linalg.norm(mid_hand - mid_hip)/shoulder_length

  return flexion_angle, mid_hip_to_hand_distance


In [ ]:

def plot_results(image, pred, joint_names, joint_edges):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib.patches import Rectangle
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image.numpy())
    for x, y, w, h, c in pred['boxes'].numpy():
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(0, 0)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-1500, 1500)
    pose_ax.set_ylim3d(2000, 5000)
    poses3d = pred['poses3d'].numpy()
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1]
    for pose3d, pose2d in zip(poses3d, pred['poses2d'].numpy()):
        for i_start, i_end in joint_edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d.T, s=2)



In [ ]:
all_tasks = ["barbell_dead_row" ,"clean_and_press", "barbell_shrug", "deadlift"]
all_cameras = [58860488, 65906101]

analysis_results = {
    'subject':     [],
    'task':        [],
    'camera':      [],
    'frame':       [],
    'flexion_exp': [],
    'dist_exp':    [],
    'flex_pose':   [],
    'dist_pose':    []}

for subject_name in fit3d_info['train_subj_names']:
  for task_name in all_tasks:
    for camera_name in all_cameras:
      frame_no = 1
      total_frames, jpg_file_name = split_videos(subject_name, task_name, camera_name, frame_no)
      experiment_parameters = cal_ground_truth_parameters(subject_name, task_name, frame_no)
      frame_length = min([experiment_parameters[2], total_frames])

      for frame_no in range(1, frame_length, 100):
        print(f'frame: {frame_no}/task: {task_name}/camera: {camera_name}/sub: {subject_name}')
        total_frames, jpg_file_name = split_videos(subject_name, task_name, camera_name, frame_no)
        experiment_parameters = cal_ground_truth_parameters(subject_name, task_name, frame_no)

        # pose estimation
        try:
          pose_estimation_parameters = estimate_3D_pose(jpg_file_name)
        except:
          pose_estimation_parameters = ['failed', 'failed']

        # Remove the file
        os.remove(jpg_file_name)

        # write results
        analysis_results['subject'].append(subject_name)
        analysis_results['task'].append(task_name)
        analysis_results['camera'].append(camera_name)
        analysis_results['frame'].append(frame_no)
        analysis_results['flexion_exp'].append(experiment_parameters[0])
        analysis_results['dist_exp'].append(experiment_parameters[1])

        analysis_results['flex_pose'].append(pose_estimation_parameters[0])
        analysis_results['dist_pose'].append(pose_estimation_parameters[1])



frame: 1/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 101/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 201/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 301/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 401/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 501/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 601/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 701/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 801/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 901/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 1001/task: barbell_dead_row/camera: 58860488/sub: s03
frame: 1/task: barbell_dead_row/camera: 65906101/sub: s03
frame: 101/task: barbell_dead_row/camera: 65906101/sub: s03
frame: 201/task: barbell_dead_row/camera: 65906101/sub: s03
frame: 301/task: barbell_dead_row/camera: 65906101/sub: s03
frame: 401/task: barbell_dead_row/camera: 65906101/sub: s03
frame: 501/task: barbell_dead_row/camera: 6

In [ ]:
import pandas as pd

df = pd.DataFrame(analysis_results)
file_name = 'analyzed_results_metras2023.csv'
df.to_csv(file_name)

destination_dir = '/content/gdrive/MyDrive/integrating new tech/validation'
shutil.move(file_name, destination_dir)
